### Suncal-Web Measurement Risk Calculator

---

In [1]:
%matplotlib ipympl
import ipywidgets as widgets

from scipy import stats
import matplotlib.pyplot as plt

import suncal
from suncal import risk
import suncalwidgets
suncal.plotting.plotstyle.update({'font.size': 9})

In [2]:
r = risk.Risk()

style = {'description_width': 'initial'}

plt.ioff()
fig = plt.figure()
plt.ion()

def update_output(change=None):
    r.set_speclimits(lower_lim.value, upper_lim.value)
    r.set_procdist(distp.get_dist())
    r.set_testdist(distt.get_dist())
    r.set_guardband(gbf.value, gbf.value)
    with tableout:
        tableout.clear_output(wait=True)
        display(r.out)
    r.out.plot_joint(plot=fig)

def on_gbmethod(change):
    if gbmethod.value == 'Target PFA':
        gbpfa.layout.visibility = 'visible'
    else:
        gbpfa.layout.visibility = 'hidden'
        
def on_gbapply(change):
    if gbmethod.value == 'RSS':
        r.calc_guardband('rss')
    elif gbmethod.value == 'Method 6':
        r.calc_guardband('dobbert')
    else:
        r.calc_guardband('pfa', pfa=gbpfa.value/100)
    gbf.value = r.get_guardband()[0]
    update_output()

tableout = widgets.Output()    
distp = suncalwidgets.DistributionWidget()
distt = suncalwidgets.DistributionWidget()
upper_lim = widgets.FloatText(value=1, description='Upper Limit', style=style)
lower_lim = widgets.FloatText(value=-1, description='Lower Limit', style=style)
distp.param1.value = 0
distp.param2.value = .51
distt.param1.value = 0
distt.param2.value = .125
gbf = widgets.FloatText(value=0, description='Absolute Guardband', style=style)
btncalc = widgets.Button(description='Calculate')
btncalc.on_click(update_output)    

gbmethod = widgets.Dropdown(options=['RSS', 'Method 6', 'Target PFA'], value='RSS', description='Method')
gbpfa = widgets.FloatText(value=2, description='Target PFA%')
gbapply = widgets.Button(description='Apply')
gbbox = widgets.VBox([gbmethod, gbpfa, gbapply])
gbaccordion = widgets.Accordion(children=[gbbox], selected_index=None)
gbaccordion.set_title(0, 'Guardbanding')
gbpfa.layout.visibility = 'hidden'
gbmethod.observe(on_gbmethod, names='value')
gbapply.on_click(on_gbapply)

ctllayout = widgets.VBox([upper_lim, lower_lim,
                       widgets.Label('Process Distribution:'),
                       distp,
                       widgets.Label('Measurement Distribution:'),
                       distt, 
                       gbf,
                       btncalc,
                       gbaccordion])

outlayout = widgets.VBox([fig.canvas, tableout])
layout = widgets.HBox([ctllayout, outlayout])
update_output()
display(layout)

---

#### Definitions

- **Upper and Lower Limit**: The defined tolerance/specification limits for acceptable product
- **Process Distribution**: The probability distribution of all products (manufactured or calibrated)
- **Measurement Distribution**: The probability distribution of the test/inspection measurement uncertainty
- **Absolute Guardband**: The amount added to the lower limit and subtracted from the upper limit to determine acceptance limits
- **Process Risk**: Probability that any product is out of limits, regardless of any inspection/test measurement
- **Specific False Accept Risk**: probability that a product is out of tolerance given the specific measured value result
- **Global False Accept Risk (PFA)**: probability that any product is falsely accepted by an inspection/test measurement
- **Global False Reject Risk (PFR)**: probability that any product is falsely rejected by an inspection/test measurement

#### Guardbanding Methods:

- **RSS**: Calculates guardband factor as $\sqrt{1-1/TUR^2}$
- **Method 6**: Calculates guardband factor as $1 - \frac{1.04 - \exp(0.38 \log(TUR) - 0.54)}{TUR}$ (See Dobbert, "A guardband strategy for managing false-accept risk")
- **Target PFA**: Numerically solve for the guardband factor that results in the desired PFA

---

#### Suncal

Suncal was developed by the Primary Standards Lab at Sandia National Laboratories. For more features and desktop version, see [https://sandiapsl.github.io](https://sandiapsl.github.io).

© 2019-2021 National Technology & Engineering Solutions of Sandia, LLC (NTESS).
Under the terms of Contract DE-NA0003525 with NTESS, the U.S. Government
retains certain rights in this software.